In [ ]:
!sudo pip3 install keras

In [ ]:
import numpy as np
import math
import re
import pandas as pd
import nltk
import string

from bs4 import BeautifulSoup
from google.colab import drive
from sklearn import tree
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from unicodedata import normalize

In [ ]:
nltk.download('stopwords')

Montamos drive para recoger los datos

In [ ]:
drive.mount('/content/drive')

# Preparación de los datos





In [ ]:
cols = ["texto", "nota"]
tweet_data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Data/alltweets.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="utf8"
)


Mostramos las 5 primeras filas del csv

In [ ]:
tweet_data.head(5)

Los tweets con nota = 1 son tweets de alto impacto, mientras que los tweets con nota = 0 son tweets de bajo impacto.



Vemos cuantos tweets hay de tipo "bueno" y cuántos de tipo "malo"

In [ ]:
print(tweet_data['nota'].value_counts())

## Limpieza del texto

In [ ]:
string.punctuation

In [ ]:
stopwords.words("spanish")

Función para limpiar el texto antes de trabajar con él

In [ ]:
def clean_data(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  #pasamos a minúsculas
  tweet = tweet.lower()
  #Eliminamos las menciones a usuarios
  tweet = re.sub(r"@[a-z0-9]+", ' ', tweet)
  #Eliminamos los enlaces
  tweet = re.sub(r"https?://[a-z0-9./]", ' ', tweet)
  #Nos quedamos con los caracteres importantes
  tweet = re.sub(r"[^a-zÀ-ÿ']", ' ', tweet)
  #Se quitan los espacios en blanco que se hayan generado de más
  tweet = re.sub(r" +", ' ', tweet)   
  #Se eliminan las tildes
  tweet = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", tweet), 0, re.I)       
  #limpiamos ahora con nltk
  notPunctuation = [c for c in tweet if c not in string.punctuation]
  notPunctuation = "".join(notPunctuation)
  words = notPunctuation.split()
  tweet = [word for word in words if word.lower() not in stopwords.words("spanish")]
  return tweet

Limpiamos los tweets

In [ ]:
tweet_data["texto"].apply(clean_data)

La columna "nota" del dataset indica si un tweet se puede considerar influyente o no. Vamos a ver cuál es el porcentaje de estos valores en el dataset original.

In [ ]:
tweet_data.groupby("nota").count()/len(tweet_data)

Hacemos división del dataset en train y en test. Se utiliza el método de sklearn train_test_split con el parámetro stratify que nos permite generar bloques el porcentaje de notas que hay en el dataset original.

##Test y Train

In [ ]:
tweet_train, tweet_test, train_value, test_value = train_test_split(tweet_data["texto"], tweet_data["nota"], test_size=0.2, random_state=38, stratify = tweet_data["nota"])

Volvemos a mostrar el porcentaje de los bloques de datos generados

In [ ]:
train_value.value_counts()/len(train_value)

In [ ]:
test_value.value_counts()/len(test_value)

#Random Forest

##Tokenización

Tokenizamos utilizando CounVectorizer

In [ ]:
transformer = CountVectorizer(analyzer=clean_data).fit(tweet_data["texto"])

In [ ]:
print(transformer.vocabulary_)

In [ ]:
print(len(transformer.vocabulary_))

##Modelo

In [ ]:
pipeline = Pipeline([
                     ('vectorizer', CountVectorizer(analyzer=clean_data)),
                     ('transformer', TfidfTransformer()),
                     ('classifier', RandomForestClassifier())
                     ])

##Entrenamiento

In [ ]:
pipeline.fit(tweet_train, train_value)

##Predicción

In [ ]:
predictions = pipeline.predict(tweet_test)


In [ ]:
print(classification_report(test_value, predictions, digits=4))